<h1 align = 'center'>Movie Recommendation</h1>
<br>
<h3 align = 'center'>Author - Naman Talwar</h3>
<br>

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv("movies.csv")
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
data.shape

(4803, 24)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [5]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [6]:
selected_features = ["genres","keywords",'tagline',"cast","director"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
for feature in selected_features:
    data[feature] = data[feature].fillna('')

In [8]:
combined_features = data["genres"]+data["keywords"]+data["tagline"]+data["cast"]+data["director"]

In [9]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [10]:
vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [12]:
print(feature_vectors)

  (0, 245)	0.07858581760804864
  (0, 444)	0.09109355212252296
  (0, 8936)	0.11806131645084655
  (0, 21836)	0.09966592997173947
  (0, 9304)	0.27385789694599244
  (0, 4456)	0.2179967521551067
  (0, 10193)	0.1653243242081299
  (0, 22916)	0.3366875640692919
  (0, 26334)	0.130597230719163
  (0, 4676)	0.24529097704249617
  (0, 22778)	0.27385789694599244
  (0, 24197)	0.07518543993419267
  (0, 27161)	0.12619886579371511
  (0, 18164)	0.08690831799482268
  (0, 18558)	0.26121683351048536
  (0, 21521)	0.15623566659433683
  (0, 27182)	0.23480088356130557
  (0, 27540)	0.19771357974524179
  (0, 21487)	0.21550128478931552
  (0, 22464)	0.2020708956871175
  (0, 26547)	0.19638671147741735
  (0, 23267)	0.16128139780622522
  (0, 14550)	0.22368093636480682
  (0, 16614)	0.1569607387745327
  (0, 20729)	0.27385789694599244
  :	:
  (4801, 18790)	0.22450140828104795
  (4801, 11776)	0.3005604008026175
  (4801, 758)	0.19369913462690624
  (4801, 8121)	0.2611505149930138
  (4801, 27513)	0.3005604008026175
  (4801, 2

In [13]:
similarity = cosine_similarity(feature_vectors)

In [14]:
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [15]:
similarity.shape

(4803, 4803)

<h3>Taking Input

In [16]:
name = input("Enter the name of your favourite movie: ")

Enter the name of your favourite movie: Avengers


In [17]:
list_of_all_titles = data['title'].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [18]:
find_close_match = difflib.get_close_matches(name,list_of_all_titles)
find_close_match

['The Avengers', 'Swingers']

In [19]:
close_match = find_close_match[0]
close_match

'The Avengers'

In [20]:
index = data[data.title==close_match]['index'].values[0]
index

16

In [21]:
similarity_score = list(enumerate(similarity[index]))
similarity_score

[(0, 0.015430503883444694),
 (1, 0.0),
 (2, 0.033207930328442936),
 (3, 0.006482452679358777),
 (4, 0.024918771275211464),
 (5, 0.006077636981019067),
 (6, 0.0),
 (7, 0.6424444374106305),
 (8, 0.0),
 (9, 0.129753120205267),
 (10, 0.017827681221460202),
 (11, 0.005490100242913768),
 (12, 0.0),
 (13, 0.005731579013364705),
 (14, 0.14044013634950722),
 (15, 0.009683526480953029),
 (16, 1.0000000000000004),
 (17, 0.005821328598807161),
 (18, 0.015014574494127454),
 (19, 0.005188994552091901),
 (20, 0.09849429292850054),
 (21, 0.022614030968416684),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.010202317448049444),
 (26, 0.27072517284276515),
 (27, 0.013958274926309423),
 (28, 0.06669106234451064),
 (29, 0.016859771882894267),
 (30, 0.10187389042229433),
 (31, 0.13233370972545233),
 (32, 0.009227840090727343),
 (33, 0.21574628614927752),
 (34, 0.0),
 (35, 0.03255470344645148),
 (36, 0.048991462469535695),
 (37, 0.0),
 (38, 0.19238862715348617),
 (39, 0.015346026961909051),
 (40, 0.0),
 (41, 0

In [22]:
len(similarity_score)

4803

In [23]:
sort = sorted(similarity_score,key = lambda x:x[1], reverse = True)
sort

[(16, 1.0000000000000004),
 (7, 0.6424444374106305),
 (85, 0.31403222285225113),
 (79, 0.30792623764880095),
 (174, 0.2901376837412583),
 (26, 0.27072517284276515),
 (511, 0.2457568003047),
 (129, 0.23357780859998825),
 (64, 0.22923948760243926),
 (203, 0.22754461810246723),
 (126, 0.21723322801194825),
 (33, 0.21574628614927752),
 (788, 0.20991954983251357),
 (182, 0.20377339938691705),
 (38, 0.19238862715348617),
 (46, 0.18596229870041542),
 (68, 0.18564323259620552),
 (4401, 0.1578729419133452),
 (14, 0.14044013634950722),
 (870, 0.1345993415810784),
 (101, 0.13428652890513654),
 (1740, 0.13361448192535913),
 (31, 0.13233370972545233),
 (782, 0.13123248761119302),
 (9, 0.129753120205267),
 (421, 0.12965944199609944),
 (2235, 0.12941186329311644),
 (94, 0.1226053076563336),
 (2197, 0.12059616201436643),
 (1720, 0.1179411474329135),
 (700, 0.1147481544696262),
 (4759, 0.10560063415361388),
 (169, 0.10237623872970075),
 (356, 0.10232776916102307),
 (30, 0.10187389042229433),
 (615, 0.1

In [24]:
print("Your Suggestions List: \n")
i = 1
for m in sort:
    ind = m[0]
    title = data[data.index==ind]['title'].values[0]
    if(i<21):
        print(i,'.',title)
        i=i+1
        

Your Suggestions List: 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Iron Man 2
5 . The Incredible Hulk
6 . Captain America: Civil War
7 . X-Men
8 . Thor
9 . X-Men: Apocalypse
10 . X2
11 . Thor: The Dark World
12 . X-Men: The Last Stand
13 . Deadpool
14 . Ant-Man
15 . The Amazing Spider-Man 2
16 . X-Men: Days of Future Past
17 . Iron Man
18 . The Helix... Loaded
19 . Man of Steel
20 . Superman II


<br>
<h3>The Recommendation System

In [26]:
name = input(' Enter your favourite movie name : ')
list_of_all_titles = data['title'].tolist()

find_close_match = difflib.get_close_matches(name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = data[data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

input_movie = sorted_similar_movies.pop(0) #removing the entered movie from output

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = data[data.index==index]['title'].values[0]
  if (i<21):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Knives Out
Movies suggested for you : 

1 . Whiteout
2 . They
3 . Jeff, Who Lives at Home
4 . Assault on Precinct 13
5 . Auto Focus
6 . Flicka
7 . The Yellow Handkerchief
8 . Thank You for Smoking
9 . The Mummy: Tomb of the Dragon Emperor
10 . Pandorum
11 . Duets
12 . The Cooler
13 . The General's Daughter
14 . Dark Water
15 . Saw
16 . Up in the Air
17 . Butterfly on a Wheel
18 . Payback
19 . Prisoners
20 . Seventh Son
